In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import PIL

In [ ]:
import cv2
from torchvision import datasets, transforms

### Датасет

In [ ]:
mnist = datasets.MNIST('./DLSCH/data/', transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))

In [ ]:
sample = mnist.train_data[np.random.randint(len(mnist.train_data))].numpy()

In [ ]:
plt.imshow(sample, cmap='gray')

### Линейные фильтры

In [ ]:
def display_gray(img):
    plt.figure(figsize=(5,5))
    plt.imshow(img, cmap='gray')

In [ ]:
def apply_filter(img, filter):
    output = []
    for i in range(img.shape[0] - filter.shape[0] + 1):
        output_row = []
        for j in range(img.shape[1] - filter.shape[1] + 1):
            output_row.append(np.maximum(np.tensordot(img[i:i+filter.shape[0], j:j+filter.shape[1]], filter), 0))
        output.append(output_row)
    return np.array(output)

In [ ]:
none_filter = np.array([[0, 0, 0],
                        [0, 1, 0],
                        [0, 0, 0]])
display_gray(apply_filter(sample, none_filter))

In [ ]:
edge_filter = np.array([[1, 1, 1],
                        [1, -8, 1],
                        [1, 1, 1]])
display_gray(apply_filter(sample, edge_filter))

In [ ]:
sharp_filter = np.array([[0, -1, 0],
                        [-1, 5, -1],
                        [0, -1, 0]])
display_gray(apply_filter(sample, sharp_filter))

In [ ]:
blur_filter = np.array([[1, 1, 1],
                        [1, 1, 1],
                        [1, 1, 1]])
display_gray(apply_filter(sample, blur_filter))

## Добавим stride

In [ ]:
def apply_filter_with_stride(img, filter, stride=1):
    output = []
    # ваш код здесь
    return np.array(output)

## PyTorch

In [ ]:
from __future__ import print_function
import torch
from torch.autograd import Variable

In [ ]:
a = torch.from_numpy(np.array([4, 1]))
b = torch.from_numpy(np.array([4, 1], dtype='float32'))
print(a)
print(b)

In [ ]:
w = torch.from_numpy(np.array([[1, 2], [2 ,1 ]]))

In [ ]:
w = Variable(w, requires_grad=True)

In [ ]:
w

In [ ]:
av = Variable(a, requires_grad=True)

In [ ]:
f = 2 * w @ av

In [ ]:
v = torch.sum(f)

In [ ]:
v

In [ ]:
v.backward()

In [ ]:
w.grad

In [ ]:
av.grad

## DenseNN

In [ ]:
class Layer(object):
    '''
        Абстрактный класс слоя
    '''
    def __init__(self, input_shape, output_shape):
        self.input_shape = input_shape
        self.output_shape = output_shape
    '''
        Прямой проход
    '''
    def forward(self, input):
        assert False, 'Abstract class'
    '''
        Обратный проход
        optimizer - объект с функцией update(params, output), обновляющий параметры
    '''
    def backward(self, optimizer):
        assert False, 'Abstract class'

In [ ]:
class Dense(Layer):
    '''
        Полносвязный слой.
    '''
    def __init__(self, input_size, output_size):
        assert input_size > 0
        assert output_size > 0
        super(Dense, self).__init__((-1, input_size), (-1, output_size))
        self.weights = #your code here
        self.bias = #your code here
    
    def forward(self, x):
        self.output = #your code here
        return self.output
    
    def backward(self, optimizer):
        params = [self.weights, self.bias]
        optimizer.update(params, self.output)

In [ ]:
class ReLU(Layer):
    '''
        ReLU
    '''
    def __init__(self, input_size):
        super(ReLU, self).__init__((-1, input_size), (-1, input_size))
    
    def forward(self,x):
        self.output = x.clamp(min = 0)
        return self.output
    
    def backward(self, optimizer):
        pass

Напоминание: $softmax(x)_{i} = \frac{e^{x_i}}{\sum_{j}e^{x_j}}$

In [ ]:
class Softmax(Layer):
    '''
        Softmax 
    '''
    def __init__(self, input_size):
        super(Softmax, self).__init__((-1, input_size), (-1, input_size))
    
    def forward(self,x):
        self.output = #your code here 
        return self.output
    
    def backward(self, optimizer):
        pass

In [ ]:
class Loss(object):
    '''
        Абстрактная фуенкция потерь
    '''
    def __init__(self):
        pass
    
    def forward(self, x, y):
        self.output = self.do_forward(x, y)
        return self.output
    
    def backward(self):
        self.output.backward()
        
    def do_forward(self, x, y):
        assert False, 'Abstract class'

In [ ]:
class CrossEntropy(Loss):
    '''
        Кросс-энтропия
    '''
    def do_forward(self, x, y):
        return torch.sum(-1*y* torch.squeeze(torch.log(x+1e-9)))

In [ ]:
class SeqNet:
    '''
        Сеть, связывающая слои последовательно
    '''
    def __init__(self):
        self.layers = []
        self.output = None
        self.loss = None
        
    def add(self, layer):
        self.layers.append(layer)
        
    def setLoss(self, loss):
        self.loss = loss
        
    def forward(self, x, y=None):
        self.output = x
        for layer in self.layers:
            self.output = layer.forward(self.output)
        if (self.loss is not None) and (y is not None):
            self.loss_value = self.loss.forward(self.output, y)
        return self.output
    
    def backward(self, optimizer):
        self.loss.backward()
        for layer in self.layers[::-1]:
            layer.backward(optimizer)

In [ ]:
class Optimizer:
    def __init__(self):
        pass
    
    def update(self, params, output):
        assert False, 'Abstract class'

In [ ]:
class SimpleOptimizer(Optimizer):
    
    def __init__(self, learning_rate):
        self.learning_rate = learning_rate
        
    def update(self, params, output):
        for param in params:
            param.data -= self.learning_rate * param.grad.data

### MNIST net

In [ ]:
import torch.utils

In [ ]:
indices = torch.from_numpy(np.random.permutation(len(mnist.train_data)))
X = mnist.train_data[indices]
y = mnist.train_labels[indices]

In [ ]:
y_one_hot = torch.FloatTensor(len(y), 10)
y_one_hot.zero_()
y_one_hot = y_one_hot.scatter_(1, y.view(-1, 1), 1)

In [ ]:
test_size = 10000
X_train = X[:-test_size].float()/255-0.5
X_test = X[-test_size:].float()/255-0.5
y_train = y_one_hot[:-test_size].float()
y_test = y_one_hot[-test_size:].float()

In [ ]:
plt.imshow(X_train[0].numpy(), cmap='gray')

In [ ]:
from tqdm import trange
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.random.permutation(len(inputs))
    for start_idx in trange(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [ ]:
from IPython.display import clear_output

In [ ]:
def accuracy(result, val_tensor):
    t, ind_result = torch.max(result, 1)
    t, ind_val = torch.max(val_tensor, 1)
    return (torch.sum((ind_result==ind_val).float())/len(y_test)).data.numpy()[0]

In [ ]:
def train(net, n_epochs, batch_size, X, y, X_test, y_test, shuffle=False):
    optimizer = SimpleOptimizer(0.5*1e-6)
    loss_history = []
    test_history = []
    x_tensor = Variable(torch.FloatTensor((batch_size, X.shape[1], X.shape[2])))
    test_tensor = Variable(X_test)
    val_tensor = Variable(y_test)
    y_tensor = Variable(torch.FloatTensor((batch_size, y.shape[1])))
    for i in range(n_epochs):
        loss_per_batch = []
        for x_batch, y_batch in iterate_minibatches(X, y, batch_size, shuffle):
            x_tensor.data=x_batch
            y_tensor.data=y_batch
            net.forward(x_tensor.view(-1, 28*28), y_tensor)
            loss_per_batch.append(net.loss_value.data.numpy()[0])
            net.backward(optimizer)
        clear_output()
        result = net.forward(test_tensor.view(-1, 28*28), val_tensor)
        test_history.append(net.loss_value.data.numpy()[0]/len(X_test))
        print('Accuracy:{}'.format(accuracy(result, val_tensor)))
        loss_history.append(np.average(loss_per_batch)/batch_size)
        plt.plot(np.arange(i+1), loss_history)
        plt.plot(np.arange(i+1), test_history)
        plt.show()

In [ ]:
net = SeqNet()

net.add(Dense(28*28, 320))

net.add(ReLU(320))

net.add(Dense(320, 10))

net.add(Softmax(10))

net.setLoss(CrossEntropy())

In [ ]:
train(net, 50, 256, X_train, y_train, X_test, y_test)

In [ ]:
net = SeqNet()

net.add(Dense(28*28, 320))

net.add(ReLU(320))

net.add(Dense(320, 128))

net.add(ReLU(128))

net.add(Dense(128, 10))

net.add(Softmax(10))

net.setLoss(CrossEntropy())

In [ ]:
train(net, 50, 256, X_train, y_train, X_test, y_test)